# Learning

In [96]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [97]:
MODEL = 'ResNet50'
batch_size=8
train_data_path = './CargoContainerImages/train/'
num_classes = 2
max_epoches = 10

In [98]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode ='categorical')

Found 147 images belonging to 2 classes.


In [99]:
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input
    model = ResNet50(weights = None, classes=num_classes)
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input
    model = VGG16(weights = None, classes = num_classes)
    output_node_name = 'predictions/Softmax'


In [100]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
19/19 [==============================] - 50s 3s/step - loss: 2.8184 - accuracy: 0.6327
Epoch 2/10
19/19 [==============================] - 46s 2s/step - loss: 1.2366 - accuracy: 0.7755
Epoch 3/10
19/19 [==============================] - 49s 3s/step - loss: 0.7062 - accuracy: 0.8503
Epoch 4/10
19/19 [==============================] - 46s 2s/step - loss: 0.3785 - accuracy: 0.9048
Epoch 5/10
19/19 [==============================] - 47s 2s/step - loss: 0.3720 - accuracy: 0.9456
Epoch 6/10
19/19 [==============================] - 46s 2s/step - loss: 0.1655 - accuracy: 0.9320
Epoch 7/10
19/19 [==============================] - 47s 2s/step - loss: 0.2049 - accuracy: 0.9388
Epoch 8/10
19/19 [==============================] - 47s 2s/step - loss: 0.1770 - accuracy: 0.9592
Epoch 9/10
19/19 [==============================] - 46s 2s/step - loss: 0.0712 - accuracy: 0.9864
Epoch 10/10
19/19 [==============================] - 47s 2s/step - loss: 0.0170 - accuracy: 0.9932


In [77]:
MODEL = 'VGG16'
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
19/19 [==============================] - 14s 761ms/step - loss: 0.8431 - accuracy: 0.8571
Epoch 2/10
19/19 [==============================] - 14s 763ms/step - loss: 0.6698 - accuracy: 0.7483
Epoch 3/10
19/19 [==============================] - 14s 761ms/step - loss: 0.3462 - accuracy: 0.8707
Epoch 4/10
19/19 [==============================] - 14s 763ms/step - loss: 0.3482 - accuracy: 0.8980
Epoch 5/10
19/19 [==============================] - 15s 763ms/step - loss: 0.0866 - accuracy: 0.9728
Epoch 6/10
19/19 [==============================] - 14s 761ms/step - loss: 0.0935 - accuracy: 0.9796
Epoch 7/10
19/19 [==============================] - 15s 766ms/step - loss: 0.0343 - accuracy: 0.9932
Epoch 8/10
19/19 [==============================] - 15s 774ms/step - loss: 0.0236 - accuracy: 0.9932
Epoch 9/10
19/19 [==============================] - 14s 763ms/step - loss: 0.0118 - accuracy: 0.9932
Epoch 10/10
19/19 [==============================] - 15s 764ms/step - loss: 0.0429 - accura

# Prediction

In [101]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
#import uff

In [102]:
MODEL = 'ResNet50'

In [103]:
if MODEL == 'ResNet50':
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    output_node_name  = 'predictions/Softmax'
#sess = K.get_session()
#K.set_learning_phase(0)
#graph = tf.get_default_graph()
model = load_model('saved_model/' + MODEL + 'saved.h5')
model.compile(loss = 'categorical_crossenthropy', optimizer = 'adam', metrics= ['accuracy'])
#output_graph_def = tf.graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['input_1', output_node_name])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [93]:
#uffmodel = uff.from_tensorflow(output_graph_def, [output_node_name], 
#                              output_filename = 'saved_model/' + MODEL + 'uff')

In [110]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import decode_predictions
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import decode_predictions
def predict_all(folder):
    for filename in os.listdir(folder):
        print("Predicting...", filename)
        fullpath = os.path.join(folder, filename)  # 폴더명과 파일명을 결합
        #print("fullpath", fullpath)  
        try:
            # 테스트할 이미지 불러오기
            img = load_img(fullpath, target_size=(224, 224)) 
            # ResNet에 입력하기 전에 이미지 전처리
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # 이미지 분류
            preds = model.predict(x)
            print('horizontal:vertical   ', preds)
            #print('Predicted:', decode_predictions(preds, top=1)[0])

        except Exception as e:
            print(e)

In [111]:
predict_all('./CargoContainerImages/train/1/')

Predicting... BHCU2088072.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU2191783.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CBCU2000317.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CCLU1026613.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CCLU2058301.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CCLU4755619.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CCLU4856213.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CICU9679964.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CICU9679964_2.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CICU9728204.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CISU2000076.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CISU2000203.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CKLU2049325.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CKLU2049330.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CKLU4104437.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CKLU4106701.jpg
horizontal:vertical    

In [112]:
predict_all('./CargoContainerImages/train/2/')

Predicting... BMOU4699433.jpg
horizontal:vertical    [[1. 0.]]
Predicting... BMOU5782049.jpg
horizontal:vertical    [[1. 0.]]
Predicting... BSIU2575064.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU2728316.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU4051570.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU8121016.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CARU3858347.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAXU8004650.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CBXU2785862.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CMAU5349828.jpg
horizontal:vertical    [[1. 0.]]
Predicting... crop002.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CRXU1525331.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DFSU7696722.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DFSU7696722_2.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DFSU7697586.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DLSU5000210.JPG
horizontal:vertical    [[1.

In [71]:
a = 86
b=61
a/(a+b)

0.5850340136054422